In [6]:
import sys
import os
from dotenv import load_dotenv


# Load .env file
load_dotenv()

# Force all HTTPS libraries to use corporate cert
os.environ["REQUESTS_CA_BUNDLE"] = os.getenv("REQUESTS_CA_BUNDLE")
os.environ["CURL_CA_BUNDLE"] = os.getenv("REQUESTS_CA_BUNDLE")
os.environ["SSL_CERT_FILE"] = os.getenv("REQUESTS_CA_BUNDLE") 
from paths import CHUNKS_JSONL_PATH, FAISS_INDEX_PATH, METADATA_JSON_PATH,NARRATIVE_DIR 

print("Chunks JSONL Path:", CHUNKS_JSONL_PATH)


Chunks JSONL Path: c:\Users\250331\Documents\GenAI Projects\EaglesGPT\data\chunks.jsonl


1. Load and Chunk the wikipedia narrative docs.

In [7]:
import os
import json

# -----------------------------------
# Step 1: Chunking Function
# -----------------------------------
def chunk_text_fixed(text, chunk_size=300, overlap=50):
    """
    Splits text into overlapping chunks.
    Each chunk has `chunk_size` words, with `overlap` words shared with the next chunk.
    """
    words = text.split()
    chunks = []
    step = chunk_size - overlap

    if step <= 0:
        raise ValueError("chunk_size must be greater than overlap")

    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks

# -----------------------------------
# Step 2: Load .txt Files, Chunk, Print Examples, Save as JSONL
# -----------------------------------
def load_and_chunk_wikipedia(folder_path, output_path, chunk_size=300, overlap=50):
    all_chunks = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            full_path = os.path.join(folder_path, filename)
            with open(full_path, 'r', encoding='utf-8') as f:
                text = f.read()
                doc_title = filename.replace('.txt', '')

                chunks = chunk_text_fixed(text, chunk_size=chunk_size, overlap=overlap)

                for i, chunk in enumerate(chunks):
                    chunk_id = f"{doc_title}_{str(i+1).zfill(3)}"
                    all_chunks.append({
                        'chunk_id': chunk_id,
                        'doc_title': doc_title,
                        'text': chunk
                    })

    # --- Print first 3 chunks ---
    print("\n--- Example Chunks (JSON) ---")
    for chunk in all_chunks[:3]:
        print(json.dumps(chunk, indent=2))

    # --- Save all chunks to JSONL ---
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as out_file:
        for chunk in all_chunks:
            out_file.write(json.dumps(chunk) + '\n')

    print(f"\n✅ Saved {len(all_chunks)} chunks to {output_path}")
    return all_chunks

# -----------------------------------
# Step 3: Setup Clean Project Paths (relative to notebook folder)
# -----------------------------------

"""NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))  # go one level up
NARRATIVE_DIR = os.path.join(PROJECT_ROOT, 'data', 'narratives')
CHUNKS_OUT_PATH = os.path.join(PROJECT_ROOT, 'data', 'chunks.jsonl')"""

# --- Run It ---
chunks = load_and_chunk_wikipedia(folder_path=NARRATIVE_DIR, output_path=CHUNKS_JSONL_PATH)



--- Example Chunks (JSON) ---
{
  "chunk_id": "2020_philadelphia_eagles_season_001",
  "doc_title": "2020_philadelphia_eagles_season",
  "text": "The 2020 season was the Philadelphia Eagles' 88th in the National Football League (NFL) and their fifth and final under head coach Doug Pederson. They failed to improve on their 9\u20137 record from the previous season following a 23\u201317 loss to the Seattle Seahawks in Week 12. They were eliminated from playoff contention for the first time since 2016 following a Week 16 loss to the Dallas Cowboys and finished with a dismal 4\u201311\u20131 record, the second-worst in the National Football Conference (NFC), and their worst since 2012. After starting 3\u20134\u20131 heading into their bye week and leading the NFC East, the Eagles would lose 7 of their last 8 games. Injuries and poor quarterback play were factors in their struggles in the season. On January 11, 2021, the Eagles announced head coach Doug Pederson would not return after the 

2. Embed the corpus and index

In [8]:
#! pip install -U sentence-transformers
#! pip install faiss-cpu
#! pip install -U bitsandbytes
#! pip install --upgrade openai
#! pip install python-dotenv

import os
import json
import numpy as np
import faiss
import re
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from tqdm import tqdm
import httpx

# -----------------------------------
# 🔑 Load API Key Securely
# -----------------------------------


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY environment variable not set.")

client = OpenAI(
    api_key=OPENAI_API_KEY,
    http_client=httpx.Client(verify=os.environ.get("REQUESTS_CA_BUNDLE"))
)



# -----------------------------------
# 📂 Paths
# -----------------------------------
"""PROJECT_ROOT = os.getcwd()  # Root of EaglesGPT repo
CHUNKS_JSONL_PATH = os.path.join(PROJECT_ROOT, 'data', 'chunks.jsonl')
FAISS_INDEX_PATH = os.path.join(PROJECT_ROOT, 'data', 'narrative_index.faiss')
METADATA_JSON_PATH = os.path.join(PROJECT_ROOT, 'data', 'narrative_metadata.json')"""

# -----------------------------------
# 📄 Load Chunks
# -----------------------------------
def load_chunks_from_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# -----------------------------------
# 🧠 Embed Chunks
# -----------------------------------
def embed_chunks(chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    texts = [chunk['text'] for chunk in chunks]
    print(f"🔁 Embedding {len(texts)} chunks...")
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
    for i, emb in enumerate(embeddings):
        chunks[i]['embedding'] = emb.tolist()
    return chunks

# -----------------------------------
# 🗃️ Build FAISS Index
# -----------------------------------
def build_faiss_index(embedded_chunks, index_path, metadata_path):
    embeddings = np.array([c['embedding'] for c in embedded_chunks]).astype('float32')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    os.makedirs(os.path.dirname(index_path), exist_ok=True)
    faiss.write_index(index, index_path)
    print(f"✅ FAISS index saved to {index_path}")

    metadata = [{k: c[k] for k in ['chunk_id', 'doc_title', 'text']} for c in embedded_chunks]
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved to {metadata_path}")

# -----------------------------------
# 🔍 Retrieve Chunks
# -----------------------------------
def extract_years(text):
    return re.findall(r'20\d{2}', text)

def retrieve_narrative_chunks(question, index_path, metadata_path, model_name='all-MiniLM-L6-v2', top_k=5):
    print(f"\n🔍 Retrieving top {top_k} chunks for question: {question}")

    target_years = set(extract_years(question))

    index = faiss.read_index(index_path)
    with open(metadata_path, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    model = SentenceTransformer(model_name)
    query_vector = model.encode([question]).astype('float32')

    distances, indices = index.search(query_vector, top_k * 3)
    candidates = [metadata[i] for i in indices[0]]

    def score(chunk):
        title = chunk['doc_title']
        return 0 if any(y in title for y in target_years) else 1

    candidates.sort(key=score)
    results = candidates[:top_k]

    print(f"\n📄 Retrieved Chunks (boosted by years: {', '.join(target_years) if target_years else 'None'}):\n")
    for i, r in enumerate(results):
        print(f"[{i+1}] ({r['chunk_id']}) from {r['doc_title']}")
        print(r['text'][:300] + "\n---\n")

    return results

# -----------------------------------
# 🤖 Answer with OpenAI
# -----------------------------------
def answer_with_openai(question, retrieved_chunks, model="gpt-3.5-turbo"):
    context = "\n\n".join(chunk["text"] for chunk in retrieved_chunks)

    system_prompt = (
        "You are a knowledgeable assistant helping summarize NFL team narratives. "
        "Use only the context provided. Do not hallucinate or include outside information."
    )

    user_prompt = f"""Answer the question based on the following Eagles season narratives:

{context}

Question: {question}
Answer:"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    answer = response.choices[0].message.content
    print("\n🤖 OpenAI's Answer:\n")
    print(answer)
    return answer

if __name__ == "__main__":
    # Step 1: Load chunks from JSONL
    chunks = load_chunks_from_jsonl(CHUNKS_JSONL_PATH)
    print(f"📦 Loaded {len(chunks)} chunks.")

    # Step 2: Embed the chunks
    embedded_chunks = embed_chunks(chunks)

    # Step 3: Build and save FAISS index + metadata
    build_faiss_index(embedded_chunks, FAISS_INDEX_PATH, METADATA_JSON_PATH)

📦 Loaded 160 chunks.
🔁 Embedding 160 chunks...


Batches: 100%|██████████| 5/5 [00:03<00:00,  1.51it/s]

✅ FAISS index saved to c:\Users\250331\Documents\GenAI Projects\EaglesGPT\data\narrative_index.faiss
✅ Metadata saved to c:\Users\250331\Documents\GenAI Projects\EaglesGPT\data\narrative_metadata.json


In [9]:


# Define the question
question = "Which was more disappointing, the 2020 or 2023 Eagles season?"

# Retrieve top relevant narrative chunks
retrieved_chunks = retrieve_narrative_chunks(
    question,
    index_path=FAISS_INDEX_PATH,
    metadata_path=METADATA_JSON_PATH
)

# Use OpenAI to generate the answer from retrieved chunks
answer = answer_with_openai(question, retrieved_chunks)

# Print the answer
print("\n🦅 Answer from OpenAI:\n")
print(answer)


🔍 Retrieving top 5 chunks for question: Which was more disappointing, the 2020 or 2023 Eagles season?

📄 Retrieved Chunks (boosted by years: 2023, 2020):

[1] (2020_philadelphia_eagles_season_001) from 2020_philadelphia_eagles_season
The 2020 season was the Philadelphia Eagles' 88th in the National Football League (NFL) and their fifth and final under head coach Doug Pederson. They failed to improve on their 9–7 record from the previous season following a 23–17 loss to the Seattle Seahawks in Week 12. They were eliminated from p
---

[2] (2023_philadelphia_eagles_season_001) from 2023_philadelphia_eagles_season
The 2023 season was the Philadelphia Eagles' 91st season in the National Football League (NFL), their 30th under the ownership of Jeffrey Lurie and their third under head coach Nick Sirianni. The Eagles entered the season as defending NFC champions. This season would mark the first season since 2010
---

[3] (2020_philadelphia_eagles_season_002) from 2020_philadelphia_eagles_se